In [1]:
!pip install -q transformers accelerate peft sentence-transformers tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from pathlib import Path
import shutil, json, os, torch, re, gc

# Paths
drive_data_path = Path('/content/drive/MyDrive/HotpotQA_snapshot/evidence_hotpotqa_entities_with_paths.json')
work_dir = Path('/content/llm_answer_generation')
work_dir.mkdir(parents=True, exist_ok=True)
local_data = work_dir / 'evidence_hotpotqa_entities_with_paths.json'

# Copy dataset locally (faster I/O)
shutil.copy2(drive_data_path, local_data)
print('Dataset ready →', local_data)

Dataset ready → /content/llm_answer_generation/evidence_hotpotqa_entities_with_paths.json


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = 'Qwen/Qwen3-14B'

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype='auto',
    device_map='auto',
    trust_remote_code=True
)
print('Model loaded on', model.device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

...

model-00008-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Model loaded on cuda:0


In [5]:
import json, re, tqdm
from typing import List, Dict

def load_json(path: Path) -> List[Dict]:
    with open(path, 'r') as f:
        return json.load(f)

def save_json(obj, path: Path):
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, 'w') as f:
        json.dump(obj, f, indent=4, ensure_ascii=False)

def _extract_answer(full_text: str) -> str:
    """Remove <think>…</think> blocks and collapse whitespace."""
    answer = re.sub(r'<think>.*?</think>', '', full_text, flags=re.S).strip()
    return re.sub(r'\s+', ' ', answer).strip()

def generate_answers(
    data_file: Path,
    save_file: Path,
    *,
    max_new_tokens: int = 128,
    temperature: float = 0.6,
    top_p: float = 0.95,
    top_k: int = 20
):
    data = load_json(data_file)

    prompt_template = (
        'Given the question, its associated knowledge‑graph paths, and evidence contexts below, '
        'please generate a concise, precise answer in English. The answer must strictly adhere to the following guidelines:\n'
        '- The answer should be directly relevant to the question.\n'
        '- Provide the answer in a clear, straightforward format.\n'
        '- Limit your answer to **no more than 10 words**, focusing on the essential information requested.\n'
        '- Use both the provided information and your own knowledge to answer as accurately as possible.\n\n'
        'QUESTION: {question}\n'
        'PATHS:\n{paths}\n'
        'CONTEXT:\n{context}\n'
        'ANSWER:'
    )

    none_template = (
        'Given the following question, create a concise (≤10 words) English answer using your own knowledge if needed.\n'
        'QUESTION: {question}\n'
        'ANSWER:'
    )

    for rec in tqdm.tqdm(data, total=len(data)):
        path_list = rec.get('Final_paths', [])
        ctx_list  = rec.get('evidence', [])

        if path_list or ctx_list:
            paths_block = '\n'.join(f'{i+1}: {p}' for i, p in enumerate(path_list))
            ctx_block   = '\n'.join(f'{i+1}: {c}' for i, c in enumerate(ctx_list))
            user_prompt = prompt_template.format(question=rec['question'], paths=paths_block, context=ctx_block)
        else:
            user_prompt = none_template.format(question=rec['question'])

        messages = [
            {'role': 'user',      'content': user_prompt},
            {'role': 'assistant', 'content': ''}
        ]

        chat_text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=True
        )

        inputs = tokenizer(chat_text, return_tensors='pt').to(model.device)

        gen_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            top_k=top_k,
            do_sample=True
        )

        new_tokens  = gen_ids[0][len(inputs.input_ids[0]):]
        full_output = tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
        short_answer = _extract_answer(full_output)

        # Append answer to each record
        rec['answer'] = short_answer
        rec['model_output'] = full_output  # optional for inspection


    save_json(data, save_file)
    print(f'Finished — saved to {save_file}')
    return data

In [7]:
answers_out = Path('/content/drive/MyDrive/HotpotQA_snapshot/hotpotqa_answers_qwen.json')

_ = generate_answers(
        local_data,
        answers_out,
        max_new_tokens=32768,
        temperature=0.6,
        top_p=0.95,
        top_k=20
)

100%|██████████| 100/100 [47:00<00:00, 28.20s/it]

Finished — saved to /content/drive/MyDrive/HotpotQA_snapshot/hotpotqa_answers_qwen.json
